### What is the causal effect of heat waves on electricity demand in regions with high penetration of solar power?

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def eda(data):
    df = data.copy()
    df = df[['Region', 'UTC time', 'Local date', 'Hour', 'Local time', 'Time zone',
                         'DF', 'D', 'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC',
                         'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH', 'CO2 Emissions Generated']]
    df['Local date'] = pd.to_datetime(df['Local date'], format='%d%b%Y')
    df['day'] = df['Local date'].dt.day
    df['month'] = df['Local date'].dt.month
    df['year'] = df['Local date'].dt.year
    df['dotw'] = df['Local date'].dt.dayofweek
    df = df[df['year'] >= 2019]
    df['Hour'] = df['Hour'] - 1
    df = df[~((df['day'] == 18) & (df['month'] == 11) & (df['year'] == 2024))]
    df = df[~((df['day'] == 17) & (df['month'] == 11) & (df['year'] == 2024))]
    df['D'] = pd.to_numeric(df['D'].str.replace(',', '', regex=False), errors='coerce')
    return df

In [5]:
california = eda(pd.read_csv('california.csv', low_memory=False))

In [6]:
california

,Region,UTC time,Local date,Hour,Local time,Time zone,DF,D,Sum (NG),NG: COL,...,NG: OIL,NG: WAT,NG: SUN,NG: WND,NG: OTH,CO2 Emissions Generated,day,month,year,dotw
30721,CAL,01Jan2019 9:00:00,2019-01-01,0,01Jan2019 1:00:00,Pacific,"26,940",26876,"18,423","1,778",...,89.0,"2,295",-22,"2,604",779,"5,341",1,1,2019,1
30722,CAL,01Jan2019 10:00:00,2019-01-01,1,01Jan2019 2:00:00,Pacific,"25,922",25989,"17,613","1,765",...,89.0,"2,002",-22,"2,514",731,"5,174",1,1,2019,1
30723,CAL,01Jan2019 11:00:00,2019-01-01,2,01Jan2019 3:00:00,Pacific,"25,159",25295,"17,235","1,743",...,88.0,"1,724",-25,"2,539",724,"5,115",1,1,2019,1
30724,CAL,01Jan2019 12:00:00,2019-01-01,3,01Jan2019 4:00:00,Pacific,"24,700",24884,"17,147","1,727",...,90.0,"1,757",-25,"2,464",727,"5,084",1,1,2019,1
30725,CAL,01Jan2019 13:00:00,2019-01-01,4,01Jan2019 5:00:00,Pacific,"24,699",24922,"17,553","1,747",...,91.0,"1,962",-26,"2,280",751,"5,245",1,1,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82244,CAL,17Nov2024 4:00:00,2024-11-16,19,16Nov2024 20:00:00,Pacific,"30,649",28841,"20,997",-1,...,46.0,"2,131",-27,"1,399","5,731","3,945",16,11,2024,5
82245,CAL,17Nov2024 5:00:00,2024-11-16,20,16Nov2024 21:00:00,Pacific,"30,125",28645,"20,434",-1,...,44.0,"2,169",-25,957,"5,322","4,042",16,11,2024,5
82246,CAL,17Nov2024 6:00:00,2024-11-16,21,16Nov2024 22:00:00,Pacific,"29,505",28340,"20,023",-1,...,46.0,"2,208",-24,793,"4,664","4,187",16,11,2024,5
82247,CAL,17Nov2024 7:00:00,2024-11-16,22,16Nov2024 23:00:00,Pacific,"28,324",27900,"19,584",-1,...,46.0,"2,241",-23,663,"4,201","4,225",16,11,2024,5
